In [1]:
import torch
if torch.cuda.is_available():
    device = torch.device("cuda")
    print(f"Using GPU: {torch.cuda.get_device_name(0)}")
else:
    device = torch.device("cpu")
    print("Using CPU")

Using GPU: Tesla T4


In [2]:
pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 57.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 27.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.4 MB/s eta 0:00:00


In [3]:
pip install sentence_transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 kB 3.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.5 MB/s eta 0:00:00
  Created wheel for sentence_transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125942 sha256=2eb6a6b569bb62f6714e57e63565a742dd6e92da528178ec3c0ae43d98742105
  Stored in directory: /root/.cache/pip/wheels/62/f2/10/1e606fd5f02395388f74e7462910fe851042f97238cbbd902f
Successfully built sentence_transformers


In [ ]:
from sentence_transformers import SentenceTransformer, util
model = SentenceTransformer('all-MiniLM-L6-v2')


In [23]:
def get_most_similar_answers(question, answers, model):
    # Encode the question and possible answers
    question_embedding = model.encode(question)
    answer_embeddings = [model.encode(answer) for answer in answers]

    # Compute cosine similarity between the question and each possible answer
    cos_sim = [util.cos_sim(question_embedding, answer_embedding) for answer_embedding in answer_embeddings]

    # Combine the answers with their cosine similarity scores
    answer_scores = list(zip(answers, cos_sim))

    # Sort the answers by their cosine similarity score in descending order
    answer_scores = sorted(answer_scores, key=lambda x: x[1], reverse=True)

    # Return the top 5 most similar answers and their cosine similarity score
    return answer_scores[:5]


In [51]:
import pandas as pd
import re

df = pd.read_csv("validation.csv")

cosine_result = []
cosine_result_score =[]
for index, row in df.iterrows():
        # Extract the paragraph from the current row
        paragraph = row["targetParagraphs"]
        # removing unnecessary stuff
        paragraph = re.sub('[\'\"/\\\n]', '', paragraph)
        paragraph = paragraph.replace("'", "")
        paragraph = paragraph.replace(" '", "")
        paragraph = paragraph.replace("' ", "")
        
        # Split the paragraph into a list of sentences using a period as the delimiter
        sentences = paragraph.split('.')
        sentences = [sentence.strip() for sentence in sentences if sentence.strip()]
        
        question = row["postText"]
        answer = row["spoiler"]

        print("---------")
        print(sentences)
        print("quesiton: " + question)
        print("answer: " + str(answer))

        result = get_most_similar_answers(question, sentences, model)

        # Print the top most similar answer and its cosine similarity score
        print("result: "+result[0][0])
        print("score: ")
        print(result[0][1].item())



        cosine_result.append(result[0][0])
        cosine_result_score.append(result[0][1].item())

        #top_answer, top_score = result[0]
        #print("Top most similar answer: {} (cosine similarity score: {:.4f})".format(top_answer, top_score.item()))



Streaming output truncated to the last 5000 lines.
answer: Female staffers adopted a meeting strategy they called "amplification": When a woman made a key point, other women would repeat it, giving credit to its author.
result: Obama noticed, she and others said, and began calling more often on women and junior aides
score: 
0.6334397792816162
---------
['The sneaky designers at Beats by Dre employ a clever trick to make you think that the company’s plastic headphones are durable products worth the premium price', 'Beats by Dre headphones are garbage', 'Besides their crappy sound, they’re basically designed to break', 'And yet they sell millions of pairs of headphones', 'It’s practically a perfect business: Take crap and sell it for a fortune', 'How do they do it? In part it’s marketing, and you know, Dr', 'Dre', 'But there’s more', 'A detailed teardown (via PopMech) of a pair of Beats’ immensely popular Solo headphones conducted by hardware-focused venture capital firm Bolt has some a

In [52]:
df['generated_spoiler'] = cosine_result
df['generated_spoiler_score'] = cosine_result_score
     
df.to_csv('calculateBleu.csv', index=False)

In [53]:
pip install bert_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [54]:
import pandas as pd
from bert_score import score

total_precision, total_recall, total_f1, count = 0, 0, 0, 0

for index, row in df.iterrows():
    if row['tags'] == "passage":
        # Convert floats to strings if necessary
        if isinstance(row['generated_spoiler'], float):
            gen_spoiler = str(row['generated_spoiler'])
        else:
            gen_spoiler = row['generated_spoiler']
        if isinstance(row['spoiler'], float):
            spoiler = str(row['spoiler'])
        else:
            spoiler = row['spoiler']

        # Calculate the BERT score for each pair of sentences
        scores = score([gen_spoiler], [spoiler], lang='en', verbose=False)

        # Extract precision, recall, and F1 scores
        precision, recall, f1 = scores

        # Add to running total
        total_precision += precision.mean()
        total_recall += recall.mean()
        total_f1 += f1.mean()
        count += 1

if count > 0:
    # Calculate and print average scores
    avg_precision = total_precision / count
    avg_recall = total_recall / count
    avg_f1 = total_f1 / count
    print(f"BERT precision score (average): {avg_precision:.4f}")
    print(f"BERT recall score (average): {avg_recall:.4f}")
    print(f"BERT F1 score (average): {avg_f1:.4f}")
else:
    print("No rows with 'tags' equal to 'phrase' found in DataFrame")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaM

BERT precision score (average): 0.8540
BERT recall score (average): 0.8656
BERT F1 score (average): 0.8594


In [79]:
import pandas as pd
from bert_score import score

total_precision, total_recall, total_f1, count = 0, 0, 0, 0

for index, row in df.iterrows():
    if row['tags'] == "phrase":
        # Convert floats to strings if necessary
        if isinstance(row['generated_spoiler'], float):
            gen_spoiler = str(row['generated_spoiler'])
        else:
            gen_spoiler = row['generated_spoiler']
        if isinstance(row['spoiler'], float):
            spoiler = str(row['spoiler'])
        else:
            spoiler = row['spoiler']

        # Calculate the BERT score for each pair of sentences
        scores = score([gen_spoiler], [spoiler], lang='en', verbose=False)

        # Extract precision, recall, and F1 scores
        precision, recall, f1 = scores

        # Add to running total
        total_precision += precision.mean()
        total_recall += recall.mean()
        total_f1 += f1.mean()
        count += 1

if count > 0:
    # Calculate and print average scores
    avg_precision = total_precision / count
    avg_recall = total_recall / count
    avg_f1 = total_f1 / count
    print(f"BERT precision score (average): {avg_precision:.4f}")
    print(f"BERT recall score (average): {avg_recall:.4f}")
    print(f"BERT F1 score (average): {avg_f1:.4f}")
else:
    print("No rows with 'tags' equal to 'phrase' found in DataFrame")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaM

BERT precision score (average): 0.7992
BERT recall score (average): 0.8281
BERT F1 score (average): 0.8127


In [80]:
import pandas as pd
from bert_score import score

total_precision, total_recall, total_f1, count = 0, 0, 0, 0

for index, row in df.iterrows():
    if row['tags'] == "multi":
        # Convert floats to strings if necessary
        if isinstance(row['generated_spoiler'], float):
            gen_spoiler = str(row['generated_spoiler'])
        else:
            gen_spoiler = row['generated_spoiler']
        if isinstance(row['spoiler'], float):
            spoiler = str(row['spoiler'])
        else:
            spoiler = row['spoiler']

        # Calculate the BERT score for each pair of sentences
        scores = score([gen_spoiler], [spoiler], lang='en', verbose=False)

        # Extract precision, recall, and F1 scores
        precision, recall, f1 = scores

        # Add to running total
        total_precision += precision.mean()
        total_recall += recall.mean()
        total_f1 += f1.mean()
        count += 1

if count > 0:
    # Calculate and print average scores
    avg_precision = total_precision / count
    avg_recall = total_recall / count
    avg_f1 = total_f1 / count
    print(f"BERT precision score (average): {avg_precision:.4f}")
    print(f"BERT recall score (average): {avg_recall:.4f}")
    print(f"BERT F1 score (average): {avg_f1:.4f}")
else:
    print("No rows with 'tags' equal to 'phrase' found in DataFrame")

Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.decoder.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.layer_norm.weight', 'lm_head.bias']
- This IS expected if you are initializing RobertaM

BERT precision score (average): 0.8319
BERT recall score (average): 0.8223
BERT F1 score (average): 0.8268


In [56]:
pip install meteor_score

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement meteor_score (from versions: none)
ERROR: No matching distribution found for meteor_score


In [60]:
pip install datasets


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 14.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 17.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 15.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 29.7 MB/s eta 0:00:00


In [61]:
pip install seqeval

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16180 sha256=48b52275a8fd2e3d27fe81c56e7cac422cbfce5cca280fdc79e66e38b8582aaa
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


In [68]:
pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [71]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [72]:
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [73]:
import nltk
from nltk.translate import meteor_score
import re

meteor_scores = []

for index, row in df.iterrows():
    if row['tags'] == "passage":
        ref = row['spoiler']
        hyp = row['generated_spoiler']

        # Check if ref and hyp are strings
        if type(ref) != str or type(hyp) != str:
            continue

        # Check if ref and hyp are empty or have length 0
        if len(ref) == 0 or len(hyp) == 0:
            continue

        # Remove non-ascii characters and special characters
        ref = re.sub(r'[^\x00-\x7F]+',' ', ref)
        hyp = re.sub(r'[^\x00-\x7F]+',' ', hyp)
        ref = re.sub(r'[^a-zA-Z0-9\s]','', ref)
        hyp = re.sub(r'[^a-zA-Z0-9\s]','', hyp)

        ref_tokens = nltk.word_tokenize(ref)
        hyp_tokens = nltk.word_tokenize(hyp)

        score = meteor_score.meteor_score([ref_tokens], hyp_tokens)
        meteor_scores.append(score)

average_meteor_score = sum(meteor_scores) / len(meteor_scores)
print(f"Average METEOR score passage: {average_meteor_score}")

Average METEOR score passage: 0.25220001601625053


In [81]:
import nltk
from nltk.translate import meteor_score
import re

meteor_scores = []

for index, row in df.iterrows():
    if row['tags'] == "phrase":
        ref = row['spoiler']
        hyp = row['generated_spoiler']

        # Check if ref and hyp are strings
        if type(ref) != str or type(hyp) != str:
            continue

        # Check if ref and hyp are empty or have length 0
        if len(ref) == 0 or len(hyp) == 0:
            continue

        # Remove non-ascii characters and special characters
        ref = re.sub(r'[^\x00-\x7F]+',' ', ref)
        hyp = re.sub(r'[^\x00-\x7F]+',' ', hyp)
        ref = re.sub(r'[^a-zA-Z0-9\s]','', ref)
        hyp = re.sub(r'[^a-zA-Z0-9\s]','', hyp)

        ref_tokens = nltk.word_tokenize(ref)
        hyp_tokens = nltk.word_tokenize(hyp)

        score = meteor_score.meteor_score([ref_tokens], hyp_tokens)
        meteor_scores.append(score)

average_meteor_score = sum(meteor_scores) / len(meteor_scores)
print(f"Average METEOR score phrase: {average_meteor_score}")

Average METEOR score phrase: 0.13249803473476757


In [85]:
import nltk
from nltk.translate import meteor_score
import re

meteor_scores = []

for index, row in df.iterrows():
    if row['tags'] == "multi":
        ref = row['spoiler']
        hyp = row['generated_spoiler']

        # Check if ref and hyp are strings
        if type(ref) != str or type(hyp) != str:
            continue

        # Check if ref and hyp are empty or have length 0
        if len(ref) == 0 or len(hyp) == 0:
            continue

        # Remove non-ascii characters and special characters
        ref = re.sub(r'[^\x00-\x7F]+',' ', ref)
        hyp = re.sub(r'[^\x00-\x7F]+',' ', hyp)
        ref = re.sub(r'[^a-zA-Z0-9\s]','', ref)
        hyp = re.sub(r'[^a-zA-Z0-9\s]','', hyp)

        ref_tokens = nltk.word_tokenize(ref)
        hyp_tokens = nltk.word_tokenize(hyp)

        score = meteor_score.meteor_score([ref_tokens], hyp_tokens)
        meteor_scores.append(score)

average_meteor_score = sum(meteor_scores) / len(meteor_scores)
print(f"Average METEOR score multi: {average_meteor_score}")

Average METEOR score multi: 0.1116750180615278


In [77]:
import pandas as pd
from nltk.tokenize import TreebankWordTokenizer

# Define the tokenizer function
tokenizer = TreebankWordTokenizer().tokenize

# Tokenize the generated spoilers
df['tokenized_generated_spoiler'] = df['generated_spoiler'].apply(tokenizer)


In [78]:
import nltk

bleu_scores = []

for index, row in df.iterrows():
    if row['tags'] == "passage":
        true_spoiler = row['tokenized_spoiler']
        myoutput = row['tokenized_generated_spoiler']
        bleu_score = nltk.translate.bleu_score.sentence_bleu([true_spoiler], myoutput)
        bleu_scores.append(bleu_score)

average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU score for passage: {average_bleu_score}")

/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Average BLEU score for passage: 5.516802141265827e-233


In [83]:
import nltk

bleu_scores = []

for index, row in df.iterrows():
    if row['tags'] == "phrase":
        true_spoiler = row['tokenized_spoiler']
        myoutput = row['tokenized_generated_spoiler']
        bleu_score = nltk.translate.bleu_score.sentence_bleu([true_spoiler], myoutput)
        bleu_scores.append(bleu_score)

average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU score for phrase: {average_bleu_score}")

Average BLEU score for phrase: 4.541742293732548e-232


In [84]:
import nltk

bleu_scores = []

for index, row in df.iterrows():
    if row['tags'] == "multi":
        true_spoiler = row['tokenized_spoiler']
        myoutput = row['tokenized_generated_spoiler']
        bleu_score = nltk.translate.bleu_score.sentence_bleu([true_spoiler], myoutput)
        bleu_scores.append(bleu_score)

average_bleu_score = sum(bleu_scores) / len(bleu_scores)
print(f"Average BLEU score for nmulti: {average_bleu_score}")

Average BLEU score for nmulti: 6.615872240991012e-233
